## Annotating the variants of genes of the patient from the gencode database

In [2]:
gene_annotations_filename = "gencode.v18.annotation.gtf"
variant_filename = 'Metabolic_variants.vcf'

In [3]:
from gtfparse import read_gtf
# returns GTF with essential columns such as "feature", "seqname", "start", "end"
# alongside the names of any optional keys which appeared in the attribute column
df = read_gtf(gene_annotations_filename)

INFO:root:Extracted GTF attributes: ['gene_id', 'transcript_id', 'gene_type', 'gene_status', 'gene_name', 'transcript_type', 'transcript_status', 'transcript_name', 'level', 'havana_gene', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'ccdsid']


In [4]:
# filter DataFrame to gene entries on chrY
df_genes = df[df["feature"] == "gene"]
df_genes.head()

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,...,transcript_status,transcript_name,level,havana_gene,tag,havana_transcript,exon_number,exon_id,ont,ccdsid
0,chr1,HAVANA,gene,11869,14412,NaN,+,0,ENSG00000223972.4,ENSG00000223972.4,...,KNOWN,DDX11L1,2,OTTHUMG00000000961.2,,,,,,
21,chr1,HAVANA,gene,14363,29806,NaN,-,0,ENSG00000227232.4,ENSG00000227232.4,...,KNOWN,WASH7P,2,OTTHUMG00000000958.1,,,,,,
82,chr1,HAVANA,gene,29554,31109,NaN,+,0,ENSG00000243485.2,ENSG00000243485.2,...,NOVEL,MIR1302-11,2,OTTHUMG00000000959.2,,,,,,
92,chr1,HAVANA,gene,34554,36081,NaN,-,0,ENSG00000237613.2,ENSG00000237613.2,...,KNOWN,FAM138A,2,OTTHUMG00000000960.1,,,,,,
100,chr1,HAVANA,gene,52473,54936,NaN,+,0,ENSG00000268020.2,ENSG00000268020.2,...,KNOWN,OR4G4P,2,OTTHUMG00000185779.1,,,,,,


In [29]:
df_genes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57445 entries, 0 to 2614559
Data columns (total 24 columns):
seqname              57445 non-null object
source               57445 non-null object
feature              57445 non-null object
start                57445 non-null int64
end                  57445 non-null int64
score                0 non-null float32
strand               57445 non-null object
frame                57445 non-null object
gene_id              57445 non-null object
transcript_id        57445 non-null object
gene_type            57445 non-null object
gene_status          57445 non-null object
gene_name            57445 non-null object
transcript_type      57445 non-null object
transcript_status    57445 non-null object
transcript_name      57445 non-null object
level                57445 non-null object
havana_gene          57445 non-null object
tag                  57445 non-null object
havana_transcript    57445 non-null object
exon_number          57445 non-nul

In [5]:
df_genes.columns

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'gene_id', 'transcript_id', 'gene_type', 'gene_status',
       'gene_name', 'transcript_type', 'transcript_status', 'transcript_name',
       'level', 'havana_gene', 'tag', 'havana_transcript', 'exon_number',
       'exon_id', 'ont', 'ccdsid'],
      dtype='object')

In [30]:
df_genes['seqname'].unique()

array(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
       'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22',
       'chrX', 'chrY', 'chrM'], dtype=object)

## Look into the list of variants and find correspondance to the gene from the gencode file

In [39]:
import vcf
vcf_reader = vcf.Reader(open(variant_filename, 'r'))

import pandas as pd
variants = pd.DataFrame(columns=['CHROM', 'start', 'end', 'strand', 'gene_id', 'transcript_id', 'gene_type', 'gene_status',
       'gene_name', 'transcript_type', 'transcript_status', 'transcript_name', 'POS','REF','ALT','FILTER'])

#CHROM	POS	ID	REF	ALT	QUAL	FILTER

In [40]:
counter = 0
for record in vcf_reader:
    chrom = record.CHROM
    pos = record.POS
    ref = record.REF
    alt = record.ALT
    filter_pass = record.FILTER
    for row in df_genes.itertuples():
        #print(str(chrom), row.seqname[3:])
        there_is_entry = str(chrom) == row.seqname[3:] and pos > row.start and pos < row.end
        if there_is_entry:
            new_row = {'POS': pos, 'start': row.start, 'end': row.end, 'strand': row.strand, 'gene_id' : row.gene_id, 'transcript_id' : row.transcript_id, 'gene_type' : row.gene_type, 'gene_status': row.gene_status, 'gene_name' : row.gene_name, 'transcript_type' : row.transcript_type, 'transcript_status': row.transcript_status, 'transcript_name' : row.transcript_name, 'CHROM' : chrom, 'POS': pos,'REF':ref,'ALT':alt,'FILTER':filter_pass}
            variants = variants.append(new_row, ignore_index=True)
            counter += 1
            #print(counter)
            if counter % 100 == 0:
                print(variants.tail())

   CHROM     start       end strand             gene_id       transcript_id  \
95     1  19197926  19229275      -  ENSG00000159423.12  ENSG00000159423.12   
96     1  19175767  19247615      -   ENSG00000255275.3   ENSG00000255275.3   
97     1  19197926  19229275      -  ENSG00000159423.12  ENSG00000159423.12   
98     1  19175767  19247615      -   ENSG00000255275.3   ENSG00000255275.3   
99     1  19197926  19229275      -  ENSG00000159423.12  ENSG00000159423.12   

         gene_type gene_status      gene_name transcript_type  \
95  protein_coding       KNOWN        ALDH4A1  protein_coding   
96  protein_coding       NOVEL  RP13-279N23.2  protein_coding   
97  protein_coding       KNOWN        ALDH4A1  protein_coding   
98  protein_coding       NOVEL  RP13-279N23.2  protein_coding   
99  protein_coding       KNOWN        ALDH4A1  protein_coding   

   transcript_status transcript_name       POS REF  ALT FILTER  
95             KNOWN         ALDH4A1  19202770   T  [C]     []  
96  

    CHROM    start      end strand            gene_id      transcript_id  \
795     2  9628615  9695921      -  ENSG00000151694.8  ENSG00000151694.8   
796     2  9628615  9695921      -  ENSG00000151694.8  ENSG00000151694.8   
797     2  9628615  9695921      -  ENSG00000151694.8  ENSG00000151694.8   
798     2  9628615  9695921      -  ENSG00000151694.8  ENSG00000151694.8   
799     2  9628615  9695921      -  ENSG00000151694.8  ENSG00000151694.8   

          gene_type gene_status gene_name transcript_type transcript_status  \
795  protein_coding       KNOWN    ADAM17  protein_coding             KNOWN   
796  protein_coding       KNOWN    ADAM17  protein_coding             KNOWN   
797  protein_coding       KNOWN    ADAM17  protein_coding             KNOWN   
798  protein_coding       KNOWN    ADAM17  protein_coding             KNOWN   
799  protein_coding       KNOWN    ADAM17  protein_coding             KNOWN   

    transcript_name      POS REF  ALT FILTER  
795          ADAM17  

     CHROM      start        end strand            gene_id      transcript_id  \
1495     2  241807896  241819919      +  ENSG00000172482.4  ENSG00000172482.4   
1496     2  241807896  241819919      +  ENSG00000172482.4  ENSG00000172482.4   
1497     2  241807896  241819919      +  ENSG00000172482.4  ENSG00000172482.4   
1498     2  241807896  241819919      +  ENSG00000172482.4  ENSG00000172482.4   
1499     2  242295658  242434256      +  ENSG00000006607.8  ENSG00000006607.8   

           gene_type gene_status gene_name transcript_type transcript_status  \
1495  protein_coding       KNOWN      AGXT  protein_coding             KNOWN   
1496  protein_coding       KNOWN      AGXT  protein_coding             KNOWN   
1497  protein_coding       KNOWN      AGXT  protein_coding             KNOWN   
1498  protein_coding       KNOWN      AGXT  protein_coding             KNOWN   
1499  protein_coding       KNOWN     FARP2  protein_coding             KNOWN   

     transcript_name        POS 

     CHROM     start       end strand             gene_id       transcript_id  \
2195     5  41870132  41871059      +   ENSG00000248668.1   ENSG00000248668.1   
2196     5  52391509  52405893      -  ENSG00000164172.14  ENSG00000164172.14   
2197     5  52391509  52405893      -  ENSG00000164172.14  ENSG00000164172.14   
2198     5  52405672  52410956      +   ENSG00000247796.2   ENSG00000247796.2   
2199     5  58264865  59817947      -  ENSG00000113448.12  ENSG00000113448.12   

                 gene_type gene_status      gene_name       transcript_type  \
2195  processed_transcript       NOVEL      OXCT1-AS1  processed_transcript   
2196        protein_coding       KNOWN          MOCS2        protein_coding   
2197        protein_coding       KNOWN          MOCS2        protein_coding   
2198             antisense       NOVEL  CTD-2366F13.1             antisense   
2199        protein_coding       KNOWN          PDE4D        protein_coding   

     transcript_status transcript_name

KeyboardInterrupt: 

In [ ]:
        loc = row.geo_location
        mask = distance(df_o['geo_location'].values, loc) < 100
        objects_nearby = df_o.loc[mask]
    # do something with this neighborhood